## Using helix_funcs

An example of how to use the helix_funcs module to process the HelixScope datafiles, and generate a master output table.

Process_files should generate a similarly named csv file in the process folder.
combine_processed 

In [1]:
import helix_funcs
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [2]:
s = gpd.read_file('./data/gadm28_adm1/gadm28_adm1.shp')
s = s.to_crs(epsg='4326')

In [3]:
fs = helix_funcs.identify_netcdf_and_csv_files()

In [5]:
# test_list = ['data/UEA_data/climate/pr/ECEARTH-R1.SWL_15.cl.pr.Apr.nc',
#             'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_2.eco.cSoil.nc'
#             ]

In [6]:
for f in tqdm(fs['nc'][0:4]):
    helix_funcs.process_file(file=f, shps=s, verbose=False)

Processing 'data/UEA_data/climate/pr/ECEARTH-R1.SWL_15.cl.pr.Apr.nc'


Processing 'data/CNRS_data/cSoil/orchidee-giss-ecearth.SWL_2.eco.cSoil.nc'


In [7]:
helix_funcs.combine_processed_results()

Made ./master_admin1.csv: 6,759 rows of data. 2 sources.


In [8]:
tmp = pd.read_csv("./master_admin1.csv")
tmp.head()

,name_0,iso,id_1,name_1,engtype_1,variable,swl_info,count,max,min,...,impact_tag,institution,model_long_name,model_short_name,model_taxonomy,is_multi_model_summary,is_seasonal,season,is_monthly,month
0,Mexico,MEX,9,Ciudad de México,Federal District,cSoil,2.0,3,9.882084,5.708925,...,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
1,Haiti,HTI,3,L'Artibonite,Department,cSoil,2.0,6,4.863313,2.137869,...,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
2,Honduras,HND,1,Atlántida,Department,cSoil,2.0,5,7.904777,5.777231,...,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
3,Honduras,HND,3,Colón,Department,cSoil,2.0,9,7.317976,5.605846,...,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN
4,Guyana,GUY,9,Upper Demerara-Berbice,Region,cSoil,2.0,11,6.398812,4.786508,...,eco,LSCE,ORCHIDEE,orchidee,orchidee-giss-ecearth,False,False,NaN,False,NaN


In [ ]:
outputs = helix_funcs.identify_netcdf_and_csv_files('processed/')

In [ ]:
outputs['csv'][-1]

In [ ]:
tmp = pd.read_csv(outputs['csv'][-1])